Tensorflow의 변수 생성에 함수를 이용하기 위해 변수 범위를 한정하는 방법을 알아봅니다.

[Tensorflow에서 DNN으로 Mnist 데이터셋 분류하기](https://comafire.github.io/2016-12-06-python-tf-dnn-mnist.html)에서 DNN의 설정 부분을 variable_scope() 를 써서 함수화 해보도록 하겠습니다.

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
%matplotlib inline

import tensorflow as tf
from tensorflow.contrib.learn.python.learn import datasets

In [2]:
mnist = datasets.mnist.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
with tf.name_scope("inputs"):
    x = tf.placeholder(tf.float32, shape=[None, 784])
    y = tf.placeholder(tf.float32, shape=[None, 10])
    keep_prob = tf.placeholder(tf.float32)

# 기존의 DNN 의 layer 설계시 weight 와 bias 를 
# variable_scope() 와 같이 사용하면 함수로 대체 가능하게 됩니다.
def fc_layer(x, shape):
    w = tf.get_variable(name="weight", shape=shape, initializer=tf.contrib.layers.xavier_initializer())
    b = tf.get_variable(name="bias", shape=[1, shape[1]], initializer=tf.contrib.layers.xavier_initializer())
    return tf.add(tf.matmul(x, w), b)

with tf.name_scope("layers"):
    with tf.variable_scope("l1"):
        # 이제 weight 와 bias 의 이름은 l1/weight 와 l1/bias 가 되어
        # 변수 이름 중복 오류 없이 함수를 통해 변수 생성이 가능하게 됩니다.
        l1 = tf.nn.relu(fc_layer(x, [784, 392]))
    with tf.variable_scope("l2"):
        l2 = tf.nn.relu(fc_layer(l1, [392, 196]))
    with tf.variable_scope("l3"):
        l3 = tf.nn.relu(fc_layer(l2, [196, 98]))
    with tf.variable_scope("l4"):
        l4 = tf.nn.relu(fc_layer(l3, [98, 49]))
        l4_dropout = tf.nn.dropout(l4, keep_prob)
    with tf.variable_scope("l5"):
        l5 = fc_layer(l4_dropout, [49, 10])

with tf.name_scope("optimize"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(l5, y))
    optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

with tf.name_scope("evaluate"):
    pred_y = tf.nn.softmax(l5)
    corr_ys = tf.equal(tf.argmax(pred_y, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(corr_ys, tf.float32))    

In [4]:
tb_cost = tf.summary.scalar("cost", cost)
tb_acc = tf.summary.scalar("acc", acc)
tb_merge = tf.summary.merge_all()

def mkdir(dir_path):
    if tf.gfile.Exists(dir_path):
        tf.gfile.DeleteRecursively(dir_path)
    tf.gfile.MakeDirs(dir_path)
        
with tf.Session() as sess:
    train_path = "../logs/tf-dnn-mnist-train"
    valid_path = "../logs/tf-dnn-mnist-valid"
    mkdir(train_path)
    mkdir(valid_path)

    train_writer = tf.train.SummaryWriter(train_path, sess.graph)
    valid_writer = tf.train.SummaryWriter(valid_path, sess.graph)

    for epoch in range(0, 10):    
        init = tf.global_variables_initializer()
        sess.run(init)    
        saver = tf.train.Saver()
        path_format = "../models/tf-dnn-mnist-%03d/"
        prev_model_path = path_format % (epoch - 1)
        prev_model_file = "%s%s" % (prev_model_path, "model")
        if tf.gfile.Exists(prev_model_path):
            saver.restore(sess, prev_model_file)

        for step in range(1000):
            train_batch_xs, train_batch_ys = mnist.train.next_batch(100)
            sess.run(optimizer, feed_dict={x: train_batch_xs, y: train_batch_ys, keep_prob: 0.5})

        train_cost, train_acc, train_summary = sess.run([cost, acc, tb_merge], feed_dict={x: mnist.train.images, y: mnist.train.labels, keep_prob: 1.0})
        train_writer.add_summary(train_summary, epoch)
        valid_cost, valid_acc, valid_summary = sess.run([cost, acc, tb_merge], feed_dict={x: mnist.validation.images, y: mnist.validation.labels, keep_prob: 1.0})
        valid_writer.add_summary(valid_summary, epoch)
        print "epoch: %d, train_cost: %s, train_acc: %s" % (epoch, train_cost, train_acc)
        print "epoch: %d, valid_cost: %s, valid_acc: %s" % (epoch, valid_cost, valid_acc)

        curr_model_path = path_format % epoch
        curr_model_file = "%s%s" % (curr_model_path, "model")
        mkdir(curr_model_path)
        saver.save(sess, curr_model_file)

    train_writer.close()
    valid_writer.close()

epoch: 0, train_cost: 0.0974743, train_acc: 0.969964
epoch: 0, valid_cost: 0.113474, valid_acc: 0.9662
epoch: 1, train_cost: 0.0513796, train_acc: 0.984527
epoch: 1, valid_cost: 0.0932428, valid_acc: 0.9754
epoch: 2, train_cost: 0.0301231, train_acc: 0.990855
epoch: 2, valid_cost: 0.0859358, valid_acc: 0.9792
epoch: 3, train_cost: 0.0330162, train_acc: 0.990909
epoch: 3, valid_cost: 0.12209, valid_acc: 0.978
epoch: 4, train_cost: 0.0166366, train_acc: 0.994782
epoch: 4, valid_cost: 0.0930478, valid_acc: 0.981
epoch: 5, train_cost: 0.0122785, train_acc: 0.996145
epoch: 5, valid_cost: 0.0982721, valid_acc: 0.981
epoch: 6, train_cost: 0.0119618, train_acc: 0.996218
epoch: 6, valid_cost: 0.107501, valid_acc: 0.9806
epoch: 7, train_cost: 0.00621274, train_acc: 0.998036
epoch: 7, valid_cost: 0.113027, valid_acc: 0.9812
epoch: 8, train_cost: 0.00983125, train_acc: 0.997055
epoch: 8, valid_cost: 0.113927, valid_acc: 0.9816
epoch: 9, train_cost: 0.0143301, train_acc: 0.996927
epoch: 9, valid_co

In [5]:
with tf.Session() as sess:
    for epoch in range(10):
        init = tf.global_variables_initializer()
        sess.run(init)

        saver = tf.train.Saver()
        
        path_format = "../models/tf-dnn-mnist-%03d/"
        curr_model_path = path_format % epoch
        curr_model_file = "%s%s" % (curr_model_path, "model")
        saver.restore(sess, curr_model_file)

        test_acc = sess.run(acc, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
        print "epoch: %d, test acc: %s" % (epoch, test_acc)

epoch: 0, test acc: 0.9641
epoch: 1, test acc: 0.9718
epoch: 2, test acc: 0.9785
epoch: 3, test acc: 0.9765
epoch: 4, test acc: 0.9791
epoch: 5, test acc: 0.9811
epoch: 6, test acc: 0.9807
epoch: 7, test acc: 0.9814
epoch: 8, test acc: 0.9803
epoch: 9, test acc: 0.9812
